In [ ]:
# example notebook

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadroshaanriaz/students-performance-dataset-cleaned")
path

/home/manu/prog/ai/kaggle-student/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 9.94k/9.94k [00:00<00:00, 17.8MB/s]

Extracting files...


'/home/manu/.cache/kagglehub/datasets/muhammadroshaanriaz/students-performance-dataset-cleaned/versions/1'

In [4]:
import pandas as pd

df = pd.read_csv(f'{path}/Cleaned_Students_Performance.csv')

In [6]:
df['race_ethnicity'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       1000 non-null   int64  
 1   race_ethnicity               1000 non-null   object 
 2   parental_level_of_education  1000 non-null   object 
 3   lunch                        1000 non-null   int64  
 4   test_preparation_course      1000 non-null   int64  
 5   math_score                   1000 non-null   int64  
 6   reading_score                1000 non-null   int64  
 7   writing_score                1000 non-null   int64  
 8   total_score                  1000 non-null   int64  
 9   average_score                1000 non-null   float64
dtypes: float64(1), int64(7), object(2)
memory usage: 78.2+ KB


In [8]:
df['race_ethnicity'].unique(), df['parental_level_of_education'].unique() 

(array(['group B', 'group C', 'group A', 'group D', 'group E'],
       dtype=object),
 array(["bachelor's degree", 'some college', "master's degree",
        "associate's degree", 'high school', 'some high school'],
       dtype=object))

In [ ]:
race_ethnicities = ['group A', 'group B', 'group C', 'group D', 'group E']
education_levels = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]

